<strong>맥주프로젝트를 사용할 맥주리스트를 크롤링하자.</strong><br>

http://www.wine21.com/13_search/beer_list.html#view<br><br>
<strong>해당 사이트를 알아낸 경위</strong><br><br>
데이터를 수집할 만한 사이트를 찾기는 쉬운 일이 아니다.<br>
하지만 항상 방법은 있었다. <br>
이번 경우에는 국내에서 판매 중인 해외맥주를 찾기 어려웠다.<br>
그래서 마트에서 직접 데이터를 수집했다. 그리고 다시 인터넷으로 마트에서 수집했던 데이터를 인터넷에 검색해봤다.<br>
만약 해당 맥주의 목록이 있는 사이트라면 다른 맥주들도 많을테니 그것을 크롤링하면 되니까!

홈페이지 구성<br>
1. 사진은 사이트 내 고유로고가 들어있다. 사용이 어려울거  같다.<br>
(만약 팀원들이 허락한다면 O.K)
2. 생산국가, 맥주이름, 가격, 맥주종류가 모두 목록화되어 있다.(크롤링하기 좋음)
    - 한글 / 영문 모두 존재한다.
3. 한 페이지 내에 총 100건의 맥주를 보여주며, 총 1317건의 맥주리스트가 있다.
    - 페이지 간 이동이 hidden으로 처리되어 1 > 2 > ... > 14 페이지를 자동으로 넘기는데 한계가 있다.

## 공통 코드

In [17]:
# a는 마지막 페이지번호를 입력
def source_crawler(page):
    # 필요한 라이브러리 임포트
    from bs4 import BeautifulSoup
    from urllib.request import urlopen, Request

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By

    from bs4 import BeautifulSoup
    import time

    # selenium을 이용하여 웹페이지 소스 긁어오기
    chrome_driver = 'C:/Users/JIn/Documents/chromedriver' 

    # 크롤링할 url
    target_url = 'https://www.wine21.com/13_search/beer_list.html#List'
    
    page = page-1
    soup = []
    
    # 첫 번째 페이지 열기
    driver = webdriver.Chrome(chrome_driver)
    driver.get(target_url)

    # 목록 수 100개 설정 
    driver.find_element_by_xpath("//select/option[@value='100']").click()

    # 페이지소스 저장
    src = driver.page_source
    soup.append(BeautifulSoup(src))
    
    driver.close()
        
    for i in range(page):

        # 페이지 열기
        driver = webdriver.Chrome(chrome_driver)
        driver.get(target_url)

        # 목록 수 100개 설정 
        driver.find_element_by_xpath("//select/option[@value='100']").click()

        # 페이지 전환
        page = driver.find_elements_by_xpath('//div[@class="pagenate ver_web"]/a')
        page[i].click()

        # 페이지소스 저장
        src = driver.page_source
        soup.append(BeautifulSoup(src))

        driver.close()

    return soup

In [58]:
def beer_mining(source):
    import pandas as pd

    생산자_kor=[]
    생산자_eng=[]
    생산지역_kor=[]
    생산지역_eng=[]
    상품명_eng=[]
    상품명_kor=[]
    맥주타입=[]
    가격=[]

    for i in range(len(source)):
        stone=source[i].findAll("div",{"class":"column_detail1"})
        for j in range(len(source[i].findAll("h4","name_ko"))):
            raw_text = stone[j].get_text()
            생산자_kor.append(stone[j].findAll("span")[0].get_text())
            생산자_eng.append(stone[j].findAll("span")[1].get_text())
            생산지역_kor.append(stone[j].findAll("span")[2].get_text())
            생산지역_eng.append(stone[j].findAll("span")[3].get_text())
            상품명_eng.append(stone[j].findAll("span")[4].get_text())
            상품명_kor.append(stone[j].findAll("h4","name_ko")[0].get_text())
            맥주타입.append(stone[j].em.get_text())
            가격.append(stone[j].findAll("span")[6].get_text())

    return pd.DataFrame(list(zip(생산자_kor, 생산자_eng,생산지역_kor,생산지역_eng,상품명_eng,상품명_kor,맥주타입,가격)),
                 columns =['생산자_kor', '생산자_eng','생산지역_kor','생산지역_eng','상품명_eng','상품명_kor','맥주타입','가격'])

In [ ]:
# source = source_crawler(12)
# dataset = beer_mining(source)

In [60]:
df = beer_mining(source)
df.to_csv('beer_list.csv', encoding='utf-8-sig')